# Mapping Geo Data

## 地図データのありか

- [地球地図日本 | 国土地理院](https://www.gsi.go.jp/kankyochiri/gm_jpn.html#gm_jpn)

- [World Geo JSON by Johan Sundstrom](https://github.com/johan/world.geo.json): 国々とアメリカの州のデータが含まれている。

- [Highmaps](https://code.highcharts.com/mapdata/) に含まれる地図情報 (SVG, GeoJSON, JavaScript) は概ねフリー。フリーでないものは赤い印がついている。これらについては著作権標示すれば利用できるらしい。著作権標示の文面はデータに書き込まれている。

## 参考文献

- [Python geopandas + Bokeh で地理情報をプロットしたい](http://sinhrks.hatenablog.com/entry/2015/07/18/215951)

    GeoPandas から取り出したポリゴン情報を使って Bokeh で描画している。

- [Walkthrough: Mapping Basics with bokeh and GeoPandas in Python](https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9)

    Bokeh の地理情報を GeoJSON に変換して Bokeh で描画している。

## Tile Provider Maps

In [1]:
import json

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import *
from bokeh.tile_providers import get_provider, Vendors
from bokeh.sampledata.sample_geojson import geojson

print(f'地図データベンダーの一覧？:\n{", ".join(list(Vendors))}')

output_notebook()

p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")

data = get_provider(Vendors.STAMEN_TERRAIN)
#print(data.attribution)
p.add_tile(data)

show(p)

地図データベンダーの一覧？:
CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TERRAIN_RETINA, STAMEN_TONER, STAMEN_TONER_BACKGROUND, STAMEN_TONER_LABELS


Loading BokehJS ...

In [2]:
data = json.loads(geojson)
for i, feature in enumerate(data['features']):
    feature['properties']['Color'] = ['blue', 'red'][i%2]

geo_source = GeoJSONDataSource(geojson=json.dumps(data))

p = figure(background_fill_color='lightgrey')
p.circle(x='x', y='y', size=15, color='Color', alpha=.7, source=geo_source)

show(p)

# 「Python geopandas + Bokeh で地理情報をプロットしたい」より

## 準備

`pip install descartes geopandas geocoder`

In [3]:
import pandas as pd
import geopandas as gpd

In [4]:
tokyo = gpd.read_file('japan/tokyo.geojson')

In [5]:
wards = tokyo.loc[tokyo['area_ja'] == '都区部']
def ward(ward_ja):
    return wards.loc[wards['ward_ja'] == ward_ja]

足立区, 目黒区, 大田区, 品川区 = [ward(name) for name in '足立区 目黒区 大田区 品川区'.split()]

In [6]:
from bokeh.transform import factor_cmap

factors = list(wards['ward_ja'])
pallet = (['#ffffff', '#ffe0e0', '#e0ffe0', '#e0e0ff'] * len(wards))[:len(wards)]

p = Plot(width=900, height=700)
glyphs = p.add_glyph(GeoJSONDataSource(geojson=wards.to_json()),
                     Patches(xs='xs', ys='ys',
                             fill_color=factor_cmap('ward_ja', palette=pallet, factors=factors),
                             line_color='#c0c0c0', line_width=.5))
p.add_tools(HoverTool(renderers = [glyphs],
                      tooltips = [("索引", "$index"), ('区', '@ward_ja'), ('コード', '@code')]))
show(p)

In [7]:
if True:
    # 低レベルAPIを用いた描画方法
    
    import shapely

    COLORS = ['#ffffff', '#fff0f0', '#f0fff0', '#f0f0ff']

    p = figure(width=900, height=700)
    patches = []
    for i, ward in wards.iterrows():
        polygons = ward['geometry']
        if isinstance(polygons, shapely.geometry.Polygon): polygons = [polygons]

        for polygon in polygons:
            longitudes, latitudes = zip(*list(polygon.exterior.coords))
            patches.append(p.patch(x=longitudes, y=latitudes,
                                   fill_color=COLORS[i % len(COLORS)], line_color='#c0c0c0', line_width=.5, name=str(wards['ward_ja'])))
    show(p)

[Aaron Bramson さんによる隣接関係の確認方法](https://gis.stackexchange.com/a/300262/157016)が秀逸

    someSpecialGeometry = GEOdataframeRow['geometry'].values[0]
    df['isNeighbor'] = df.apply(lambda row: row['geometry'].touches(someSpecialGeometry), axis=1)

In [19]:
def touches_目黒区(ward):
    return 目黒区['geometry'].values[0].touches(ward.geometry)

目黒区_neighbors = []
for i, ward in wards.iterrows():
    if touches_目黒区(ward): 目黒区_neighbors.append(ward)
[ward['ward_ja'] for ward in 目黒区_neighbors]

# この方法で隣接関係を構築できるので、4色問題を解けば地図の塗り分けができる。
# ヒント: Z3

['品川区', '渋谷区', '世田谷区', '大田区']

# 番外編

## Google Map

例題には Google Map を利用する方法が記載されていた。2018年ごろに Google は運用を変更し、 API key を用いたアクセスのみを許すようになった。設定方法は以下の例題中のコメント内に記載されている。

使い方はさほど難しくないようだが、授業のためには Open Street Map の方がアクセスが容易なので、そちらを採用した。

In [9]:
from bokeh.plotting import gmap

if False:
    map_options = GMapOptions(lat=30.2861, lng=-97.7394, map_type="roadmap", zoom=11)

    # For GMaps to function, Google requires you obtain and enable an API key:
    #
    #     https://developers.google.com/maps/documentation/javascript/get-api-key
    #
    # Replace the value below with your personal API key:
    p = gmap("GOOGLE_API_KEY", map_options, title="Austin")

    source = ColumnDataSource(
        data=dict(lat=[ 30.29,  30.20,  30.29],
                  lon=[-97.70, -97.74, -97.78])
    )

    p.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=source)

    show(p)

## GeoJSON を用いた描画方法

Rebecca Weng は、GeoPandas から GeoJSON を介して Bokeh で描画する簡易な方法を紹介している。残念ながら tokyo データについては、この方法はブラウザの暴走をもたらした。JavaScript 側の描画ルーチンにメモリーリークがあるのかもしれない。

In [10]:
# https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

if False:
    geosource = GeoJSONDataSource(geojson = tokyo.to_json())

    # Create figure object.
    p = figure(title = 'Lead Levels in Water Samples, 2018', 
               plot_height = 600 ,
               plot_width = 950, 
               toolbar_location = 'below',
               tools = 'pan, wheel_zoom, box_zoom, reset')
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    # Add patch renderer to figure.
    states = p.patches('xs','ys', source = geosource,
                       fill_color = None,
                       line_color = 'gray', 
                       line_width = 0.25, 
                       fill_alpha = 1)
    # Create hover tool
    p.add_tools(HoverTool(renderers = [states],
                          tooltips = [('State','@NAME'),
                                    ('Population','@POPESTIMATE2018')]))
    show(p)

## 地名から座標の入手

[平成22年度 全国道路・街路交通情勢調査（道路交通センサス）](http://www.mlit.go.jp/road/census/h22-1/index.html) より、[東京都の箇所別基本表](http://www.mlit.go.jp/road/census/h22-1/data/xls/kasyo13.xls)をダウンロードして利用している。

ここでは全国道路交通センサスデータに含まれる地点のうち、所在地が「東京都」で始まる地点の交通量を収集している。

In [12]:
L = dict(location=28, traffic=35)
locations = pd.read_excel('kasyo13.xls', skiprows=list(range(9)), header=None, index_col=False)[[L['location'], L['traffic']]]
locations.columns = L.keys()
locations = locations.dropna(subset=['location'])
locations = locations[locations['location'].str.startswith('東京都')]
locations

,location,traffic
22,東京都中央区八丁堀３丁目,16441.0
23,東京都中央区八丁堀１丁目,12780.0
24,東京都中央区新富１丁目,11569.0
25,東京都中央区銀座１丁目,14259.0
26,東京都中央区築地１丁目,14144.0
...,...,...
214,東京都足立区鹿浜２丁目,28891.0
215,東京都足立区入谷７丁目,27306.0
217,東京都江東区有明１丁目,1493.0
318,東京都江戸川区大杉3丁目,10383.0


これらの地点の名称から地理的位置を推定する機能が geocoder である。この機能を提供するライブラリは `GeoPandas`, `geopy`, `geocoder` などさまざまがあるが、日本国内の地名についてはデータベースが不十分な気がする。Yahoo! や Google の geocoder API を用いるべきだろう。

In [13]:
import geocoder

for place in ['東京工業大学', '東京都中央区八丁堀３丁目']:
    g = geocoder.osm(place, timeout=5)  # Open street map
    print(g.latlng, g.address)

[35.514745649999995, 139.48399813369707] 東京工業大学, 厚木街道, 長津田町, 緑区, 町田市, 神奈川県, 226-0026, 日本 (Japan)
None None
